## Tarea Grande 2

### Integrantes

- Elías Sabja (18639925)
- Diego Bustamante (18636713)

### Imports 

In [2]:
import pandas as pd 
import altair as alt
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import neighbors
# Si estamos un Jupyter Notebook, habilitamos el notebook para visualizar gráficos de Altair
# alt.renderers.enable('notebook')
# Quitamos la restricción de máximo de puntos a graficar
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

### Cargar y limpiar datos (1 pto)

In [4]:
## Leer archivo csv y separar los datos de la clase

data = pd.read_csv("https://raw.githubusercontent.com/Exploratorio-DCC-PUC/Syllabus/master/Tareas/TG2/astro_data.csv")

## Quitar columnas no relacionadas (0.3)

dataf = data.drop(data.columns[[0,1,2,3,11,12,13]], axis=1)

## Resolver valores nulos (0.3)

del dataf['x_filter']
dataf=dataf.dropna(subset=['i_filter']) 

## Separar los datos de las clases (0.1)

clases=dataf['class']
features=dataf.drop(dataf.columns[[6]],axis=1)

## Normalizar las características (0.3)

features["redshift"]=(features["redshift"]-features["redshift"].min())/(features["redshift"].max()-features["redshift"].min())
features["u_filter"]=(features["u_filter"]-features["u_filter"].min())/(features["u_filter"].max()-features["u_filter"].min())
features["g_filter"]=(features["g_filter"]-features["g_filter"].min())/(features["g_filter"].max()-features["g_filter"].min())
features["r_filter"]=(features["r_filter"]-features["r_filter"].min())/(features["r_filter"].max()-features["r_filter"].min())
features["i_filter"]=(features["i_filter"]-features["i_filter"].min())/(features["i_filter"].max()-features["i_filter"].min())
features["z_filter"]=(features["z_filter"]-features["z_filter"].min())/(features["z_filter"].max()-features["z_filter"].min())

HTTPError: HTTP Error 404: Not Found

### Crear y evaluar clasificador (1.5 ptos)

In [ ]:
## Separar test y train (0.3)

features_train, features_test, clases_train, clases_test = train_test_split(features,clases, test_size=0.2, random_state=1)

## Instanciar y entrenar clasificador (0.3). Busca el mejor parámetro para su clasificación

n5=KNeighborsClassifier(n_neighbors=5,weights='uniform',p=1)
n5.fit(features_train,clases_train)

## Calcular score en set de train y set de test (0.3)

clases_expect=clases_train
clases_pred=n5.predict(features_train)
print("Score de Entrenamiento:",n5.score(features_train,clases_train))
print("Score de Prueba:",n5.score(features_test,clases_test))

## Dibujar matriz de confusión para set de test (0.6)

print("\n\t \t \033[1m Score per Class \033[0m")
print(classification_report(clases_expect, clases_pred, target_names=['GALAXY', 'QSO', 'STAR'], digits=3))
matrix=confusion_matrix(clases_expect,clases_pred)
data_matrix=pd.DataFrame({'Prediction':['GALAXY','QSO','STAR','GALAXY','QSO','STAR','GALAXY','QSO','STAR'],
                          'Expected':['GALAXY','GALAXY','GALAXY','QSO','QSO','QSO','STAR','STAR','STAR'],
                          'Q':[matrix[0][0],matrix[0][1],matrix[0][2],matrix[1][0],matrix[1][1],matrix[1][2],matrix[2][0],matrix[2][1],matrix[2][2]]})
alt.Chart(data_matrix).encode(
    x='Prediction:O',
    y='Expected:O'
).properties(title='Confusion Matrix', height=250, width=250).mark_text(baseline='middle').encode(text='Q:Q')

### Evaluar datos usando clustering (1 pto)

In [ ]:
## Hacer reducción de dimensionalidad usando PCA (0.2)

pca=PCA(n_components=2).fit_transform(features)
pca_data=pd.DataFrame(pca, columns=['x','y'])
pca_km=pca_data.copy()
clases_data=pd.DataFrame(clases, columns=['class']).reset_index(drop=True)
pca_data[['class']]=clases_data[['class']]

## Visualizar los resultados obtenidos por PCA usando las clases reales (0.3)

graf1=alt.Chart(pca_data).mark_point().encode(
    x='x',
    y='y',
    color='class'
)

## Predecir los labels usando KMeans (0.2)

clustering = KMeans(n_clusters=3).fit(pca)
cluster_labels = pd.DataFrame(data=clustering.labels_, columns=['labels']).replace(0, "GALAXY").replace(1, "STAR").replace(2, "QSO")
pca_km['label'] = cluster_labels['labels']

## Visualizar los resultados usando las clases obtenidas por KMeans (0.3)

graf2=alt.Chart(pca_km).mark_point().encode(
    x='x',
    y='y',
    color='label'
)
graf1|graf2

### Informe (2.5 ptos)


### 1. ¿Eliminaste columnas en el dataset que no aportaban información? De ser así, ¿cuáles fueron y por qué? ¿Cómo resolviste el tema de los valores nulos?

Sí, eliminamos las que no aportaban información porque podrían causar sesgo. En este caso, eliminamos objid que eran los identificadores, julian_date que es la fecha, ra y dec que se refieren a la posición, camcol que se refiere a la seción de información, field que hace referencia a ubicación en una ventana y plate que es la placa que detectó el objeto. Los nulos también pueden sesgar, ellos se encuentran en 2 columnas: 'x_filter' con 86.508 filas con datos nulos de un total de 108.070 filas por lo que la eliminamos completamente e 'i_filter' con 4.323 filas del total las cuales fueron eliminadas. Así, tenenos un total de 103.747x7 datos.


### 2. ¿Qué normalizaste, filas o columnas? ¿Por qué? ¿Para qué sirve normalizar los datos?  

Normalizamos por columnas, porque ellas agrupan las diferentes características (que son los parámetros que permiten distinguir las clases) y cada una se normaliza según sus mínimos y máximos. Se normalizan los datos, para que ninguna característica o columna pese más que la otra a la hora de calcular qué clase de objeto es (si se calcula la distancia, por ejemplo, los datos más grandes o dispersos pesarán más). En nuestro caso, decidimos re-escalar todos los datos a valores entre el 0 y el 1.


### 3. ¿Por qué se separan los datos en set de entrenamiento y set de pruebas? ¿Qué proporción de los datos utilizaste para cada uno y por qué?

Porque si utilizamos los datos con que entrenamos al modelo para testearlo, sólo repetirá lo que sabe para cada objeto obteniendo una score del 100%. Luego, si intentamos utilizarlo con datos nuevos fallará en una proporción mucho mayor. Después de haber leído, probado proporciones y observado las consecuencias en la matriz de confusión utilizaremos un 80% para entrenamiento y un 20% para las pruebas. Lo anterior porque la cantidad de datos es considerablemente grande, el score del test es alto y, por último, nos aseguramos de tener 20.000 datos para las pruebas y no sea sólo suerte un buen score en test.  


### 4. ¿Qué parámetros modificaste para probar tu clasificador? ¿Cuál te dio mejor resultado y por qué crees que es así?

Modificamos la forma en cómo se sacaba la distancia entre los vecinos. Usamos Manhattan, Minkowski y Euclidiana, la que mejor resultado dio fué Manhattan. Luego, modificamos la cantidad de vecinos que toma en cuenta el modelo. Primero aumentaba el score de test hasta los 4 vecinos y luego disminuía. Después elegimos un número impar y no múltiplo de 3 cercano para que no hayan empates entre las clases al calcular los vecinos cercanos. Finalmente, utilizamos 5 vecinos y Manhattan a la hora de calcular la distancia.


### 5. Explica la diferencia entre el score obtenido en el set de entrenamiento y el set de pruebas. Justifica.

El score de entrenamiento es afectado por factores existentes dentro del modelo, por ejemplo, que este aprenda datos de memoria y en la validación esto aumentaría la predicción. En cambio, con los datos de prueba (todos nuevos) esto no es posible por lo que el score es menor.

### 6. Usando la matriz de confusión que obtuviste en la segunda parte. Identifica la clase que mejor predice el modelo. Además identifica el par de clases que suele confundirse más.

La mejor clase que predice el modelo es "GALAXY" con un f1-score de 0.981, luego "STAR" con un f1-score de 0.976 y, finalmente, "QSO" con 0.954. Además de la  matriz se utilizó la siguiente importación "from sklearn.metrics import classification_report" para calcular las scores.


### 7. Al usar PCA reducimos la dimensionalidad de los datos. Nombre dos razones para querer hacer eso.

Una razón es para ver gráficamente qué tan cerca están los grupos de datos y así ver cuáles podrían parecerse. Una segunda razón es para elegir algunas pocas características con mucho peso que podrían explicar bien un fenómeno muy complejo.


### 8. ¿Qué representa el valor K en el algoritmo KMeans? ¿Qué valor usaste para K y por qué?

El valor de K representa la cantidad de clusters que debe buscar el modelo. Usamos K=3 porque sabemos que hay 3 clases de objetos en nuestros datos.


### 9. En la tercera parte, compara el gráfico obtenido usando la información de las clases con el gráfico que no utiliza la información de las clases (KMeans). ¿Por qué crees que KMeans falla en identificar perfectamente los clusters reales?

Primero, por cercanía es muy posible que se confundan unas clases con otras. Además, el algoritmo de KMeans intenta equiparar la cantidad de elementos por cada grupo, por eso, el gráfico se asemeja a un gráfico de torta equitativo.

### Bonus (0.5 ptos)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
tree= DecisionTreeClassifier(random_state=0,max_depth=18).fit(features_train,clases_train)
clasestree_pred=tree.predict(features_test)
clasesknn_pred=n5.predict(features_test)
print("Score Tree:",tree.score(features_test,clases_test))
print("Score KNN:",n5.score(features_test,clases_test))

Notamos que, luego de ajustar los parámetros de KNN para obtener el mejor resultado,  con Decision Tree Classifier (después de ajustar el parámetro max_depth) logramos mejores resultados en el test. Esto puede ocurrir principalmente porque la distancia que utiliza KNN para comparar los sets de datos y clasificarlos no sea tan efectivo, al menos en este caso, en comparación a la red de decisiones que se utiliza en Decision Tree Classifier en base a las características de los datos que se entregaron. 